# Homework part 2

### Preparation

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time

In [2]:
from recursion import recursive_action
from recursion import check_visits
from recursion import count_visits
from recursion import last_visit
from functions import check_file_path
from functions import check_dir_path
from functions import print_data_frame

In [3]:
project_path = os.path.dirname(os.path.realpath('__file__'))
input_path = project_path + '\\input'
processed_path = project_path + '\\processed'
output_path = project_path + '\\output'

dir_paths = [
    input_path,
    processed_path,
    output_path,
]

recursive_action(iter(dir_paths), check_dir_path)
        
costs_path = input_path + '\\Costs.csv'
orders_path = input_path + '\\Orders.csv'
relations_path = input_path + '\\Relations.csv'
users_path = input_path + '\\Users.csv'
visits_path = input_path + '\\Visits.csv'

file_paths = [
    costs_path, 
    orders_path,
    relations_path,
    users_path, 
    visits_path,
]


recursive_action(iter(file_paths), check_file_path)

print('All files were successfully found!')

All files were successfully found!


In [4]:
Costs = pd.read_csv(costs_path)
Costs.__name = 'Costs'
Orders =  pd.read_csv(orders_path)
Orders.__name = 'Orders'
Relations =  pd.read_csv(relations_path)
Relations.__name = 'Relations'
Users =  pd.read_csv(users_path)
Users.__name = 'Users'
Visits = pd.read_csv(visits_path, nrows = 1000)
Visits.__name = 'Visits'
print('Files: \n Costs.csv \n Orders.csv \n Relations.csv \n Users.csv \n Visits.csv (partially)\nwere loaded')

C:\Users\mchau\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Files: 
 Costs.csv 
 Orders.csv 
 Relations.csv 
 Users.csv 
 Visits.csv (partially)
were loaded


In [7]:
Costs['date_created'] = pd.to_datetime(Costs['date_created'])
Users['Reg_date'] = pd.to_datetime(Users['Reg_date'])
Orders['Order Date'] = pd.to_datetime(Orders['Order Date'])

Orders['Order Date'] = Orders['Order Date'].dt.floor('d')
Costs['date_created'] = Costs['date_created'].dt.floor('d')
Users['Reg_date'] = Users['Reg_date'].dt.floor('d')

## Task 2
### Revenue

In [9]:
cols = ['id_user','Order Date']
order_max = Orders[cols].groupby('id_user', as_index = False).max()

In [16]:
order_period = pd.merge(Users, order_max, how='right', left_on = 'id', right_on = 'id_user')
order_period = order_period.assign(period=lambda x: x['Order Date'] - x.Reg_date)
order_period.head()

,id,Reg_date,id_partner,name,id_user,Order Date,period
0,7254798,2017-11-14,tf_usa_desk,%7Bcampaign.id%7D,7254798,2017-11-14,0 days
1,7217878,2017-11-12,tf_usa_mob,%7Bcampaign.id%7D,7217878,2017-12-26,44 days
2,5043120,2017-06-25,ADC_mobile,05b6fd5aee3e7e9249eb9eb994d6e949,5043120,2017-08-27,63 days
3,4830509,2017-06-15,W4_mobile,100019,4830509,2017-08-06,52 days
4,7149422,2017-11-06,W4_non_adult_valid,100084,7149422,2017-11-06,0 days


In [17]:
order_period = pd.merge(order_period, Relations, how='outer', on=['name', 'id_partner'])
order_period = order_period[['period', 'chanel']]
order_period.head()

,period,chanel
0,0 days,TF VB
1,44 days,TF VB Mob
2,63 days,ADC
3,52 days,W4_mobile
4,0 days,W4


In [19]:
order_period.period =(order_period.period.fillna(0)/ np.timedelta64(1, 'D')).astype(int)
order_period.head()

,period,chanel
0,0,TF VB
1,44,TF VB Mob
2,63,ADC
3,52,W4_mobile
4,0,W4


In [24]:
order_period.groupby('chanel', as_index = False).mean()
display(order_period.head())

revenue_lifetime_path =  output_path + '\\revenue_lifetime.csv'
order_period.to_csv(revenue_lifetime_path)

,period,chanel
0,0,TF VB
1,44,TF VB Mob
2,63,ADC
3,52,W4_mobile
4,0,W4
